In [ ]:
from diffusers import StableDiffusionPipeline
from PIL import Image
import numpy as np
import subprocess
# pip install backgroundremover

def image_generate(prompt):
    model_path = "/root/autodl-tmp/stable-diffusion-2-1-base"
    pipe = StableDiffusionPipeline.from_pretrained(model_path)
    pipe = pipe.to("cuda")
    
    views = [", front view", ", side view"]
    processed_images = []
    
    for view in views:
        prompt_with_view = prompt + view
        image = pipe(prompt_with_view).images[0]
        image_path = prompt_with_view.replace(" ", "_") + ".png"
        image.save(image_path)
        output_path = image_path.replace(".png", "_remove.png")
        remove_background(image_path, output_path)
        processed_images.append(output_path)

    return processed_images

def remove_background(input_path, output_path):
    subprocess.run(["backgroundremover", "-i", input_path, "-o", output_path], check=True)

def calculate_ratio(image_path):
    image = Image.open(image_path)
    if image.mode != 'RGBA':
        image = image.convert('RGBA')
    
    width, height = image.size
    pixels = image.load()
    
    min_x, max_x, min_y, max_y = width, 0, height, 0
    
    for x in range(width):
        for y in range(height):
            _, _, _, a = pixels[x, y]
            if a > 128:
                min_x, max_x = min(min_x, x), max(max_x, x)
                min_y, max_y = min(min_y, y), max(max_y, y)
    
    bbox_width = max_x - min_x + 1
    bbox_height = max_y - min_y + 1
    aspect_ratio = bbox_width / bbox_height
    
    return aspect_ratio

def generate_normalized_init_params(prompt):
    processed_images = image_generate(prompt)
    
    ratios = [calculate_ratio(image_path) for image_path in processed_images]
    ratios.append(1)  # Adding the constant 1 as per the final vector requirement
    
    vector = np.array(ratios)
    normalized_vector = vector / np.linalg.norm(vector)
    
    print("Normalized init parameters:", normalized_vector)
    return normalized_vector

# Example usage
prompt = "A sports car"
init_params = generate_normalized_init_params(prompt)
